# Influxdb notebook for SDS-sensenet

Before getting started, make sure to install the 'InfluxDB-client' library:

`pip3 install git+https://github.com/influxdata/influxdb-client-python.git`

In [1]:
import requests
import pandas as pd
import influxdb

from datetime import datetime
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

Modify 'hostname', 'user' and 'passwd' with your credentials, not mine :0)

In [4]:
hostname = 'sensors.unixjazz.org'
user = 'admin' 
passwd = '9x7f_4E11!c+Abz2'
client = influxdb.DataFrameClient(hostname, 8086, ssl=True, username=user, password=passwd)

We are now connected, so the **fun** will start. First, let's query for the available DBs:

In [5]:
client.query("show databases;")

ResultSet({'('databases', None)': [{'name': 'sample_database'}, {'name': '_internal'}, {'name': 'sensornet_db'}, {'name': 'sensenet_coords'}]})

In [6]:
coords = client.query("select * from coords", database="sensenet_coords")
df_coords = coords['coords']
df_coords = df_coords.reset_index(drop=True)
df_coords

,box,latitude,longitude,type,uid
0,sbox11,38.029883,-78.484047,outdoor,LF
1,sbox10,38.030613,-78.479057,outdoor,CCI
2,sbox9,38.025759,-78.469295,outdoor,Jon K.
3,sbox8,38.047450,-78.483030,outdoor,Angela O.
4,sbox7,38.025570,-78.516610,outdoor,Lane R.
5,sbox6,38.025092,-78.503586,outdoor,Jimmy H.
6,sbox5,38.045801,-78.503947,outdoor,Francis R.
7,sbox4,38.027580,-78.515191,outdoor,Alex N.
8,sbox3,38.024780,-78.484580,outdoor,Lucas A.
9,sbox2,38.065460,-78.495820,outdoor,Zihao Z.


In [7]:
client.query("show measurements", database="sensornet_db")

ResultSet({'('measurements', None)': [{'name': 'mqtt_consumer'}, {'name': 'sds'}, {'name': 'sds-sensenet'}, {'name': 'sensenet-test'}, {'name': 'sensenet-test-proto'}, {'name': 'user_coords'}]})

In [8]:
complete_df = client.query("select * from \"sensenet-test\"", database="sensornet_db")
df = complete_df['sensenet-test']
df[['dev_id','pm25', 'pm10', 'co2', 'temp', 'humidity', 'gtw_id', 'rssi', 'snr']].head()

,dev_id,pm25,pm10,co2,temp,humidity,gtw_id,rssi,snr
2020-02-03 00:36:03.779007152+00:00,sbox6,0,0,144,12.52,62.89,sds-gateway-3,-14,9.00
2020-02-03 00:39:38.171639705+00:00,sbox11,0,0,567,19.44,58.00,sds-gateway-1,-61,11.25
2020-02-03 00:39:57.440747505+00:00,sbox10,1,1,491,16.99,51.67,center-for-civic-innovation,-69,10.50
2020-02-03 00:40:39.755830804+00:00,sbox3,0,0,1247,19.29,37.97,sds-gateway-1,-60,12.25
2020-02-03 00:41:41.145261920+00:00,sbox4,0,0,365,17.36,43.69,sds-alex-apt,-45,8.50


Now, if we want to query for just a certain portion of the dataset or a particular time slice: (remember the time is in [RFC3339](https://docs.influxdata.com/influxdb/v1.7/query_language/data_exploration/#specify-a-time-range-with-epoch-timestamps))

In [9]:
timeslice = client.query("select \"co2\" FROM \"sensenet-test\" \
                         WHERE \"dev_id\" = 'sbox11' \
                         AND time >= '2020-02-01' AND time <= '2020-02-20 12:30:00'", \
                         database="sensornet_db")
timeslice

defaultdict(list, {'sensenet-test':                                       co2
             2020-02-03 00:39:38.171639705+00:00   567
             2020-02-03 00:54:38.936284266+00:00   565
             2020-02-03 01:09:37.977429403+00:00   569
             2020-02-03 01:24:38.046949854+00:00   600
             2020-02-03 01:39:37.975148414+00:00  4781
             ...                                   ...
             2020-02-20 03:27:06.408776830+00:00   507
             2020-02-20 03:42:05.419127045+00:00   510
             2020-02-20 03:57:05.334527023+00:00   520
             2020-02-20 04:12:05.340350912+00:00   530
             2020-02-20 04:27:05.377206814+00:00   520
             
             [1546 rows x 1 columns]})

Or, to query for all the sensorboxes with regular expressions + more intuitive time slicers!

One could use now() - 1h(our), 1w(eek), etc.

In [10]:
timeslice2 = client.query("select * FROM \"sensenet-test\" \
                         WHERE \"dev_id\" =~ /./ \
                         AND time > now() - 6w", \
                         database="sensornet_db")
ts_dt = timeslice2['sensenet-test']
ts_dt[['pm25', 'pm10', 'dev_id']].head()

,pm25,pm10,dev_id
2020-02-03 00:36:03.779007152+00:00,0,0,sbox6
2020-02-03 00:39:38.171639705+00:00,0,0,sbox11
2020-02-03 00:39:57.440747505+00:00,1,1,sbox10
2020-02-03 00:40:39.755830804+00:00,0,0,sbox3
2020-02-03 00:41:41.145261920+00:00,0,0,sbox4


You might want to filter out indoor from outdoor sensors with a regular expression: for sbox-[0-9]i sensors:

In [11]:
timeslice3 = client.query("select * FROM \"sensenet-test\" \
                              WHERE \"dev_id\" =~ /sbox-/ \
                              AND time >= now() - 1w", \
                              database="sensornet_db")
ts3_df = timeslice3['sensenet-test']
ts3_df[['pm25', 'pm10', 'dev_id']].head()

,pm25,pm10,dev_id
2020-02-23 21:58:44.793694120+00:00,2,2,sbox-i0
2020-02-23 21:59:44.693634360+00:00,1,1,sbox-i0
2020-02-23 22:00:44.676907135+00:00,1,1,sbox-i0
2020-02-23 22:01:44.731460038+00:00,1,1,sbox-i0
2020-02-23 22:02:44.640079917+00:00,1,1,sbox-i0


Now, it is time for some Panda magic: concatenate the tables with location, volunteer, and sensor values for all of the outdoor sensors (except for **!~** indoor sensors):

In [12]:
timeslice4 = client.query("select * FROM \"sensenet-test\" \
                              WHERE \"dev_id\" !~ /sbox-/ \
                              AND time >= now() - 30m", \
                              database="sensornet_db")

# get the dataframe with columns we want:
ts4_df = timeslice4['sensenet-test']
ts4_df = ts4_df[['pm25', 'pm10', 'dev_id']]
ts4_df

,pm25,pm10,dev_id
2020-02-25 01:17:54.676650051+00:00,0,0,sbox11
2020-02-25 01:19:53.338955382+00:00,0,0,sbox3
2020-02-25 01:20:13.983304991+00:00,0,0,sbox9
2020-02-25 01:21:13.632717193+00:00,0,0,sbox10
2020-02-25 01:22:28.174216206+00:00,0,0,sbox8
2020-02-25 01:26:07.537640382+00:00,0,0,sbox7
2020-02-25 01:28:09.832773970+00:00,10,10,sbox5
2020-02-25 01:32:54.640915353+00:00,0,0,sbox11
2020-02-25 01:34:53.322218166+00:00,0,0,sbox3
2020-02-25 01:35:13.970032128+00:00,0,0,sbox9


We have PM values for all sensors within a particular time slice (30 minutes). Now, we need to perform the merging with the table with geographic coordinates. Pandas to the rescue!

In [13]:
# new variable to manipulate LAT/LONG table:
new_coords = df_coords

# shift index to the left, so we do not lose timestamps during merge:
ts4_df = ts4_df.reset_index(level=0).rename(columns={'index': 'timestamp'})

# merge dataframes using dev_id from two different columns, 'box' and 'dev_id':
merged_df = pd.merge(ts4_df, new_coords[['box', 'latitude', 'longitude', 'uid']], \
                     left_on='dev_id', \
                     right_on='box',   \
                     how='left')

merged_df
final_result = merged_df.drop(columns='box')
final_result

,timestamp,pm25,pm10,dev_id,latitude,longitude,uid
0,2020-02-25 01:17:54.676650051+00:00,0,0,sbox11,38.029883,-78.484047,LF
1,2020-02-25 01:19:53.338955382+00:00,0,0,sbox3,38.024780,-78.484580,Lucas A.
2,2020-02-25 01:20:13.983304991+00:00,0,0,sbox9,38.025759,-78.469295,Jon K.
3,2020-02-25 01:21:13.632717193+00:00,0,0,sbox10,38.030613,-78.479057,CCI
4,2020-02-25 01:22:28.174216206+00:00,0,0,sbox8,38.047450,-78.483030,Angela O.
5,2020-02-25 01:26:07.537640382+00:00,0,0,sbox7,38.025570,-78.516610,Lane R.
6,2020-02-25 01:28:09.832773970+00:00,10,10,sbox5,38.045801,-78.503947,Francis R.
7,2020-02-25 01:32:54.640915353+00:00,0,0,sbox11,38.029883,-78.484047,LF
8,2020-02-25 01:34:53.322218166+00:00,0,0,sbox3,38.024780,-78.484580,Lucas A.
9,2020-02-25 01:35:13.970032128+00:00,0,0,sbox9,38.025759,-78.469295,Jon K.


Influx has an unusual syntax... so when querying for fields you must use single quotes ' ' (otherwise the queries fail). We also have to escape the double quotes **inside** the query we want to run. To be happy, be syntax-safe out there! The world is a troubled spot.

Enjoy environmental monitoring!

`-- sign-off: LFRM, 02-24-2019`